In [1]:
#MAE271D Project
# MPC

from algames import ALGAMES
from mpc_noisy import MPC_noisy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter

In [2]:
# M players
M = 3

# N horizon length
N = 10

# n state size
n = 4  # (x, y, v_x, v_y)

# m control input size
m = 2  # (a_x, a_y)

# timestep
dt = 0.1 

# collision avoidance radius
r = 0.33

# running cost matrix
Q = np.eye(n)

# terminal cost matrix
Qf = np.eye(n) * 100

# control cost matrix
R = np.eye(m)

# penalty gain
rho = 1

# penalty schedule
gamma = 10

# convergence tolerance
eps = 1e-5

# max iterations
max_iter = 20

# multiplier to enforce dynamics
dynamics_mult = 100

# (gaussian) noise mean
mean = 0

# (gaussian) noise standard deviation
sigma = 0.05

In [3]:

# ### GENERAL EXAMPLE ICs###
# # # initial state
# x1_0 = np.array([0, 0, 1, 1.75])
# x2_0 = np.array([0, -0.5, 1, 2])
# x3_0 = np.array([0.5, 0.25, 0.5, 1])
# list_x0 = [x1_0, x2_0, x3_0]

# # final state
# x1_f = np.array([1.5, 0, 2, 0])
# x2_f = np.array([2, 0, 2, 0])
# x3_f = np.array([2.5, 0, 2, 0])
# list_xf = [x1_f, x2_f, x3_f]

### LANE MERGE ICs ###
# initial state
x1_0 = np.array([1, 0, 1.5, 0])
x2_0 = np.array([0.1, 0, 1.5, 0])
x3_0 = np.array([0.1, 0.5, 2, 0])
list_x0 = [x1_0, x2_0, x3_0]

# final state
x1_f = np.array([3, 0, 2, 0])
x2_f = np.array([2, 0, 2, 0])
x3_f = np.array([2.5, 0, 2, 0])
list_xf = [x1_f, x2_f, x3_f]


# constraints
constraint_wall_y = 1
constraint_u_x_max = 5
constraint_u_y_max = constraint_u_x_max

In [ ]:
X_mpc, U_mpc = MPC_noisy(
        M,  # number players
        N,  # horizon
        n,  # state size
        m,  # control input size
        dt,  # timestep
        r,  # collision avoidance radius
        list_x0,  # list of initial states
        list_xf,  # list of target states
        Q,  # running cost matrix
        Qf,  # terminal cost matrix
        R,  # control cost matrix
        rho,  # constraint penalty value
        gamma,  # constraint penalty schedule
        eps,  # convergence tolerance
        constraint_wall_y,  # y value of horizontal wall
        constraint_u_x_max,  # control input x bound
        constraint_u_y_max,  # control input y bound
        mean,
        sigma,
        max_iter,
        dynamics_mult,
    )

In [ ]:
x_players = np.split(X_mpc, M)

# safety radius
th = np.linspace(0, 2 * np.pi, 180)
c_cos = np.cos(th) * r
c_sin = np.sin(th) * r

# plot
fig = plt.figure()
for i, x in enumerate(x_players):
    x = np.reshape(x, (n, N+1), order="F")
    plt.plot(x[0, :], x[1, :], "o--")

    # plot cola radius
    plt.plot(x[0, 0] + c_cos, x[1, 0] + c_sin, ":", c=f"C{i}")
    plt.plot(x[0, -1] + c_cos, x[1, -1] + c_sin, ":", c=f"C{i}")
plt.axhline(constraint_wall_y, c="r")
plt.gca().set_aspect("equal")
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title("Position")

In [ ]:
player1 = np.reshape(x_players[0], (n, N+1), order="F")
player2 = np.reshape(x_players[1], (n, N+1), order="F")
player3 = np.reshape(x_players[2], (n, N+1), order="F")

# animation of trajectory
fig = plt.figure()
writer = PillowWriter(fps = 3)

plt.axhline(constraint_wall_y, c="r")
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title("Position")
plt.xlim([-0.5, 3.0])
plt.ylim([-0.5, 1.1])

line1, = plt.plot([], [], "o--", color = "tab:blue")
line2, = plt.plot([], [], "o--", color = "tab:orange")
line3, = plt.plot([], [], "o--", color = "tab:green")

line1_x = []
line1_y = []
line2_x = []
line2_y = []
line3_x = []
line3_y = []
      
with writer.saving(fig, "mpc_noisy.gif", 300):
        for i in range(N):
                line1_x.append(player1[0,i])
                line1_y.append(player1[1,i])
                line2_x.append(player2[0,i])
                line2_y.append(player2[1,i])
                line3_x.append(player3[0,i])
                line3_y.append(player3[1,i])

                line1.set_data(line1_x, line1_y)
                line2.set_data(line2_x, line2_y)
                line3.set_data(line3_x, line3_y)

                # plot cola radius @ beginning & end
                if (i == 0) or (i==N-1):
                        plt.plot(line1_x[i] + c_cos, line1_y[i] + c_sin, ":", color = "tab:blue")
                        plt.plot(line2_x[i] + c_cos, line2_y[i] + c_sin, ":", color = "tab:orange")
                        plt.plot(line3_x[i] + c_cos, line3_y[i] + c_sin, ":", color = "tab:green")
                
                plt.gca().set_aspect("equal")

                writer.grab_frame()

In [ ]:
# determine if merged or not
# define "merged" as within a  1/5 radius of desired y - final position for all players

merged_tol = 1/10


x_f_small = [x1_f[1], x2_f[1], x3_f[1]]
merged = 0
for i, x in enumerate(x_players):
    x = np.reshape(x, (n, N+1), order="F")
    x_f = x[:,-1][1:2]
    if abs(x_f - x_f_small[i]) < merged_tol:
        merged = merged + 1
    else:
        merged = merged + 1
    # make sure all M players have merged
    if merged == M:
        merged = 1

print(merged)

In [5]:
# monte carlo it up
num_sims = 10
merged_arr = np.zeros(num_sims)
merged_tol = 1/5

xfs = np.empty



for j in range (num_sims):

    # run mpc - noise generated randomly inside MPC_noisy
    X_mpc, U_mpc = MPC_noisy(
            M,  # number players
            N,  # horizon
            n,  # state size
            m,  # control input size
            dt,  # timestep
            r,  # collision avoidance radius
            list_x0,  # list of initial states
            list_xf,  # list of target states
            Q,  # running cost matrix
            Qf,  # terminal cost matrix
            R,  # control cost matrix
            rho,  # constraint penalty value
            gamma,  # constraint penalty schedule
            eps,  # convergence tolerance
            constraint_wall_y,  # y value of horizontal wall
            constraint_u_x_max,  # control input x bound
            constraint_u_y_max,  # control input y bound
            mean,
            sigma,
            max_iter,
            dynamics_mult,
        )


    x_players = np.split(X_mpc, M)
    
    # # store xfs
    # xfs[i]  = x[:,-1][1:2]

    x_f_small = [x1_f[1], x2_f[1], x3_f[1]]
    merged = 0
    for i, x in enumerate(x_players):
        x = np.reshape(x, (n, N+1), order="F")
        x_f = x[:,-1][1:2]
        if abs(x_f - x_f_small[i]) < merged_tol:
            merged = merged + 1
        else:
            merged = merged + 1
        # make sure all M players have merged
        if merged == M:
            merged = 1

    merged_arr[j] = 1

print(merged_arr)



0 y_max=2.999 rho=1 C_vio=-0.049
1 y_max=2.999 rho=10 C_vio=-0.049
2 y_max=2.999 rho=100 C_vio=-0.049
3 y_max=2.999 rho=1000 C_vio=-0.049
4 y_max=2.999 rho=10000 C_vio=-0.049
5 y_max=2.999 rho=100000 C_vio=-0.049
6 y_max=2.999 rho=1000000 C_vio=-0.049
7 y_max=2.999 rho=10000000 C_vio=-0.049
8 y_max=2.999 rho=100000000 C_vio=-0.049
0 y_max=3.001 rho=1 C_vio=-0.052
1 y_max=3.001 rho=10 C_vio=-0.052
2 y_max=3.001 rho=100 C_vio=-0.052
3 y_max=3.001 rho=1000 C_vio=-0.052
4 y_max=3.001 rho=10000 C_vio=-0.052
5 y_max=3.001 rho=100000 C_vio=-0.052
6 y_max=3.001 rho=1000000 C_vio=-0.052
7 y_max=3.001 rho=10000000 C_vio=-0.052
0 y_max=3.001 rho=1 C_vio=-0.054
1 y_max=3.001 rho=10 C_vio=-0.054
2 y_max=3.001 rho=100 C_vio=-0.054
3 y_max=3.001 rho=1000 C_vio=-0.054
4 y_max=3.001 rho=10000 C_vio=-0.054
5 y_max=3.001 rho=100000 C_vio=-0.054
6 y_max=3.001 rho=1000000 C_vio=-0.054
7 y_max=3.001 rho=10000000 C_vio=-0.054
0 y_max=3.002 rho=1 C_vio=-0.056
1 y_max=3.002 rho=10 C_vio=-0.056
2 y_max=3.002 rh

In [ ]:
# velocity
plt.figure()
for i, x in enumerate(x_players):
    x = np.reshape(x, (n, N+1), order="F")
    ax = plt.subplot(211)
    ax.plot(x[2, :], "o--", c=f"C{i}")
    plt.ylabel('$v_x$')
    ax = plt.subplot(212)
    ax.plot(x[3, :], "x--", c=f"C{i}")
    plt.ylabel('$v_y$')
plt.xlabel('time horizon index')
plt.suptitle("Velocity")

In [ ]:
u_players = np.split(U_mpc, M)

# plot
plt.figure()
for i, u in enumerate(u_players):
    u = np.reshape(u, (m, N+1), order="F")
    ax = plt.subplot(211)
    plt.plot(u[0, 1:], "o--", c=f"C{i}")
    plt.hlines((constraint_u_x_max, -constraint_u_x_max), 0, N, "r")
    plt.ylabel('$a_x$')
    ax = plt.subplot(212)
    plt.plot(u[1, 1:], "x--", c=f"C{i}")
    plt.hlines((constraint_u_y_max, -constraint_u_y_max), 0, N, "r")
    plt.ylabel('$a_y$')
plt.xlabel('Time Horizon Index')
plt.suptitle("Control input")